In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>


# readings at Manchester UK in 
select ?name ?dateTime ?fahrenheit ?celsius where {
    BIND(clmr:3334099999 as ?stat)
    ?stat rdfs:label ?name .
    
    SERVICE <http://44.214.5.75:8080/sparql> {
        ?reading clmo:station ?stat ;
            clmo:dateTime ?dateTime ;
            clmo:celsius ?celsius ;
            clmo:fahrenheit ?fahrenheit .
        FILTER(?dateTime > "1985-12-30T13:20:00"^^xsd:dateTime)
    } 
} 
order by ASC(?dateTime)


In [ ]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>


select * where {
    BIND(clmr:78259399999 as ?stat)
    SERVICE <http://44.214.5.75:8080/sparql> {
        clmr:78259399999 a ?type ;
    } 
} 



In [80]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

In [65]:
%%sparql
PREFIX clmo: <http://climate.aws.com/ontology/>
PREFIX clmr: <http://climate.aws.com/resource#>


select * where {

    SERVICE <http://44.214.5.75:8080/sparql> {
        clmr:74063099999 a clmo:WeatherStation ;
        ?p ?o .
    } 
} 

In [82]:
%%sparql

select * where {
?s ?p ?o .
} limit 10